In [29]:
#librerie necessarie

import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import PyPDFLoader
from dotenv import load_dotenv
from langchain_core.runnables import Runnable, RunnablePassthrough, RunnableParallel, RunnableLambda
from IPython.display import JSON
import json
from unstructured_client import UnstructuredClient
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError
from unstructured.partition.html import partition_html
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import dict_to_elements, elements_to_json


In [ ]:

# Carica le variabili di ambiente dal file .env
load_dotenv()

# Ora puoi accedere alle variabili di ambiente
openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
#llm studio

model = ChatOpenAI(
    base_url = "http://172.17.0.1:1234/v1",
    temperature = 0,
    api_key = "not-needed",
    model_name ="llama3",
)

In [30]:
#ollama

model = ChatOpenAI(
    base_url = "http://localhost:11434/v1",
    temperature = 0,
    api_key = "not-need",
    model_name ="llama3",
)

In [ ]:
#openai

model = ChatOpenAI(
    emperature = 0,
     model_name ="gpt-4o-mini",
)

In [31]:
# Percorso della cartella contenente i PDF
folder_path = 'pdfs'

# Lista per memorizzare tutti i documenti caricati
all_documents = []

# Iterare su tutti i file nella cartella
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(folder_path, filename)
        loader = PyPDFLoader(file_path=pdf_path)
        documents = loader.load()
        all_documents.extend(documents)

# Verifica e stampa il contenuto delle pagine
for doc in all_documents:
    if hasattr(doc, 'page_content'):
        print(doc.page_content)

# Funzione per formattare i documenti
def format_documents(documents):
    return "\n\n".join(doc.page_content for doc in documents if hasattr(doc, 'page_content'))

Algorithm of Thoughts: Enhancing Exploration of Ideas
in Large Language Models
Bilgehan Sel, Ahmad Al-Tawaha, Vanshaj Khattar, Ruoxi Jia, and Ming Jin
Virginia Tech
Abstract
Current literature, aiming to surpass the “Chain-of-Thought”
approach, often resorts to an external modus operandi in-
volving halting, modifying, and then resuming the genera-
tion process to boost Large Language Models’ (LLMs) rea-
soning capacities. This mode escalates the number of query
requests, leading to increased costs, memory, and computa-
tional overheads. Addressing this, we propose the Algorithm
of Thoughts —a novel strategy that propels LLMs through
algorithmic reasoning pathways, pioneering a new mode of
in-context learning. By employing algorithmic examples, we
exploit the innate recurrence dynamics of LLMs, expand-
ing their idea exploration with merely one or a few queries.
Our technique outperforms earlier single-query methods and
stands on par with a recent multi-query strategy that employs
an e

In [ ]:
len(all_documents)

In [ ]:
print(all_documents)

In [ ]:
print(all_documents[0])

In [ ]:
all_documents[16].metadata

In [ ]:
len(all_documents[0].page_content)

In [ ]:
# Stampare i nomi dei file PDF caricati
print("PDF caricati:")
for pdf in all_documents:
    print(pdf)

In [ ]:
print(all_documents[0].page_content[0:1000])

In [ ]:
all_documents=testo

In [ ]:
# Assumi che pdfd sia già definita e carica i documenti una volta

# Carica il PDF una volta
loader = PyPDFLoader(all_documents)
documents = loader.load()

# Assicurati che ogni documento abbia l'attributo 'page_content'
for doc in documents:
    print(doc.page_content)

# Funzione per formattare i documenti
def format_documents(documents):
    return "\n\n".join(doc.page_content for doc in documents)

In [32]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size =500,
    chunk_overlap=200
)
splits =text_splitter.split_documents(all_documents)
for sp in splits:
    if (len(sp.page_content) <100):
        splits.remove(sp)

In [15]:
print ("Numero di chunks:",len(splits))

Numero di chunks: 2584


In [ ]:
print("Metadata :", splits[0].metadata)
print("Dimenzione del chunk:", len(splits[1].page_content), "caratteri")
print("Contenuto del chunk: \n" )
print(splits[1].page_content)

In [33]:
#Select Embeddings model questo funziona

from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

In [34]:
cartella="algoritmo3"

In [35]:
import os
from langchain_community.vectorstores import FAISS

if os.path.exists(cartella):
    #Carica indice FAISS cartella corrente attuale
    faiss_index=FAISS.load_local(
        cartella,
        embeddings,
        allow_dangerous_deserialization=True
    )
else:
    #crea indice FAISS dei chunk nella cartella attuale
    faiss_index = FAISS.from_documents(
        splits,
        embeddings
    )
    faiss_index.save_local(cartella)


In [36]:
retriever = faiss_index.as_retriever(

    search_type="similarity",
    search_kwargs={"k": 4} 
)

In [37]:
from langchain import hub
prompt =hub.pull("rlm/rag-prompt")

In [38]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
prompt = ChatPromptTemplate.from_template("Sei un assistente preciso e attento ; Rispondi a questa domanda in italiano: {question}, considra il seguente contesto {context}.")

In [39]:
#def format_documents(all_documents):
  #  return "\n\n".join(doc.page_contente for doc in all_documents)

rag_chain = (
    {
        "context": retriever | format_documents,
        "question": RunnablePassthrough()
        
    } 
    |prompt
    |model
    |StrOutputParser()
    
    
)

In [40]:
def query(query):
    answer = rag_chain.invoke(query)
    return answer

def queryStream(query):
    for chunk in rag_chain.stream(query):
        print(chunk, end="", flush=True)

In [41]:
query("che cosa è un prompt")

"In questo contesto, un prompt è una frase o un insieme di parole che vengono fornite a un'intelligenza artificiale (LLM) per ottenere risultati ragionevoli. In particolare, i prompt sono utilizzati per guidare la risposta dell'LLM e influire sulla sua capacità di comprendere il contesto e di generare testi coerenti.\n\nNel caso specifico menzionato nel testo, i prompt 1 e 2 sono stati utilizzati per ottenere risposte brevi e concise. Il prompt 2, in particolare, è stato formulato in modo tale da richiedere alla LLM di fornire una risposta molto breve (di solito 1-2 frasi) e non proseguire con ulteriori punti.\n\nIn generale, i prompt possono essere utilizzati per vari scopi, come ad esempio:\n\n* Guidare la risposta dell'LLM verso un certo tipo di contenuto o stile\n* Fornire informazioni specifiche che l'LLM deve considerare nella sua risposta\n* Influenzare il tono e lo stile della risposta\n* Aiutare l'LLM a comprendere il contesto e a generare testi coerenti\n\nIn questo senso, i 

In [ ]:
queryStream("Dammi esempi di prompt efficaci")